## 라이브러리

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
from scipy import stats
from sklearn.ensemble import RandomForestRegressor

## 데이터 불러오기

In [ ]:
path = './data/'

df = pd.read_csv(path+'월별데이터 총병합.csv')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)

## Categorical:

- Nominal(variables that have two or more categories, but which do not have an intrinsic order.)

    - Region_Name : 자치구 명
    - Building_Use : 건물 용도
    
- Ordinal(variables that have two or more categories just like nominal variables. Only the categories can also be ordered or ranked.)

    
## Numeric:

- Discrete
    - Year : 년
    - Month : 월
    - Region_Code : 자치구 코드
    - JS_Count : 전세 거래량
    - Sell_Count : 매매 거래량
    - School_Counts : 자치구 내 초중고 수
    - Subway_Counts : 자치구 내 지하철역 수
    - Univ_Counts : 자치구 내 대학교 수
    - Park_Counts : 자치구 내 공원 수
   
- Continous
    - Sell : 매매
    - Sell_Price : 매매 가격
    - Sell_BA = Sell_building Area : 매매 건물 면적
    - Sell_PPA = Sell_Price Per Area : 면적 당 매매 가격
    - Sell_PPP = Sell_Price Per Pyeong : 평 당 매매 가격
    - JS : 전세
    - JS_Price : 전세 가격
    - JS_BA = JS_Building Area : 임대 면적
    - JS_PPA = JS_Price Per Area : 임대 면적 당 전세 가격
    - JS_PPP = JS_Price Per Pyeong : 평 당 전세 가격
    - CR = Charter Rate : 전세가율
    - CR_PPA  = Charter_Rate_Price Per Area : 면적 당 전세가율
    - CR_PPP = Charter Rate_Price Per Pyeong : 평 당 전세가율
    - lR = Interest Rate : 금리
    - UR = Unemployment Rate : 실업률
    - LC_index = Leading Composite index : 선행종합 지수
    - CA_index = Comprehensive Accompany index : 동행종합 지수
    - TC_index = Trailing Composite index : 후행종합 지수
    - SDT_index = Supply and Demand Trend index = 전세수급동향 지수
    

## PCC

In [ ]:
# 결과를 저장할 DataFrame 생성
result_df = pd.DataFrame(columns=['Column_Name', 'PCC', 'p-value'])

# 'JS_Price'와 다른 열 간의 PCC 및 p-value 계산 및 저장
for column in df.columns:
    if column != 'JS_Price':
        if df[column].dtype != object:
            correlation, p_value = pearsonr(df['JS_Price'], df[column])
            result_df = result_df.append({'Column_Name': column, 'PCC': correlation, 'p-value': p_value}, ignore_index=True)

In [ ]:
result_df.sort_values(by='PCC', ascending=False).reset_index(drop=True)

In [ ]:
a = result_df.sort_values(by='PCC', ascending=False).reset_index(drop=True)['Column_Name'][0:13].tolist()

In [ ]:
a

In [ ]:
b = result_df.sort_values(by='PCC', ascending=False).reset_index(drop=True)['Column_Name'][20:].tolist()

In [ ]:
b

In [ ]:
df.columns[0]

## ANOVA (continous vs discrete)

In [ ]:
def anova(df):
    for column in df.columns:
        if np.issubdtype(df[column].dtype, np.integer):
            # 이 코드는 정수형 열에 대해서만 분산분석을 수행합니다.
            f_statistic, p_value = stats.f_oneway(*[group for name, group in df.groupby(column)['JS_Price']])
            print(column)
            print("F-statistic:", f_statistic, "p-value:", p_value)

In [ ]:
anova(df)

## KCC (continous vs categorical)

- 범주형 변수 더미화

In [ ]:
# 범주형 변수 더미화 함수, 범주형 변수의 범주 레벨 간의 관계가 중요할 시 사용
def oh_encoding(df):
    # DataFrame의 복사본을 만듭니다.
    encoded_df = df.copy()
    for column in df.columns:
        if df[column].dtype == object:
            encoded_df = pd.get_dummies(encoded_df, columns=[column], prefix=column)
            print(column)
    return encoded_df

df_encoded = oh_encoding(df)

In [ ]:
# 결과를 저장할 DataFrame 생성
result_df = pd.DataFrame(columns=['Column_Name', 'KCC', 'p-value'])

# 'JS_Price'와 다른 열 간의 Kendall 상관 계수 및 p-value 계산 및 저장
for column in df_encoded.columns:
    if column != 'JS_Price':
        if column.startswith('Region') or column.startswith('Building'):  
            kendall_corr, p_value = kendalltau(df_encoded['JS_Price'], df_encoded[column])
            result_df = result_df.append({'Column_Name': column, 'KCC': kendall_corr, 'p-value': p_value}, ignore_index=True)

In [ ]:
result_df.sort_values(by='KCC', ascending=False).reset_index(drop=True)

- p-value값 고려
    - PCC 결과 a=0.01일 때 UR과 SDT_index, School_Counts는 통계적으로 상관관계가 유의하지 않으므로 변수에서 제외한다.
    - Anova 결과 a=0.01일 때 Month는 통계적으로 상관관계가 유의하지 않으므로 변수에서 제외한다.
    - KCC 결과 a=0.01일 때 Region_Name_영등포구, Region_Name_동작구, Region_Name_강동구, Region_Name_성동구, Region_Name_서대문구, Building_Use_연립다세대 등은 통계적으로 상관관계가 유의하지 않으므로 변수에서 제외한다.
- correlation값 고려
    - PCC결과 상관계수가 0.1이하인 변수 제외 -> Region_Code, Crime_Rates, Month, UR, SDT_index, CR_PPA, CR_PPP, School_Counts, Univ_Counts, Park_Counts를 변수에서 제외
    - KCC결과 상관계수가 0.1이하인 변수 제외 -> Building_Use_아파트, Region_Name_강남구, Region_Name_서초구, Building_Use_단독다가구만을 변수에 포함